In [7]:
%pip install --upgrade -q aiobotocore

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.42 requires botocore==1.31.42, but you have botocore 1.31.17 which is incompatible.
boto3 1.28.42 requires botocore<1.32.0,>=1.31.42, but you have botocore 1.31.17 which is incompatible.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [63]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
import time
import json
import io
from io import StringIO
import base64
import pprint
import re

from sagemaker.image_uris import retrieve

sess = sagemaker.Session()
# write_bucket = sess.default_bucket()
# write_prefix = "fraud-detect-demo"

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

sagemaker_role = sagemaker.get_execution_role()


# # S3 locations used for parameterizing the notebook run
read_bucket = "sagemaker-codesentinel"
read_prefix = "malicious-contract-classifier"
model_prefix = "bbmodel_classify"
checkpoint = "checkpoint-300"

# read_bucket = "sagemaker-sample-files"
# read_prefix = "datasets/tabular/synthetic_automobile_claims" 
# model_prefix = "models/malicious-contract-detector"

# data_capture_key = f"{write_prefix}/data-capture"

# # S3 location of trained model artifact
model_uri = f"s3://{read_bucket}/{read_prefix}/{model_prefix}/{checkpoint}/model.tar.gz"

# # S3 path where data captured at endpoint will be stored
# data_capture_uri = f"s3://{write_bucket}/{data_capture_key}"


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [64]:
# retrieve inference image

from sagemaker.huggingface.model import HuggingFaceModel

inference_image = retrieve(
    framework="huggingface",
    region="us-east-1",
    version="4.26.0",
    image_scope="inference",
    base_framework_version="pytorch1.13.1",
instance_type="ml.m5.xlarge")

malicious_classifier = HuggingFaceModel(
    model_data=model_uri,
    role=sagemaker_role,
    transformers_version="4.26",
    pytorch_version="1.13.1",
    py_version="py39",
    image_uri=inference_image
)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [65]:
# deploy model 

predictor = malicious_classifier.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="classify-malicious-contract"
)

data = {
   "inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}

# request
predictor.predict(data)

----!

[{'label': 'Benign', 'score': 0.9977884292602539}]

In [66]:

data = {
    "inputs": ["test", "test1"]
}
predictor.predict(data)

[{'label': 'Benign', 'score': 0.9977877140045166},
 {'label': 'Benign', 'score': 0.9981564879417419}]

In [6]:
predictor.delete_endpoint()


In [22]:
from sagemaker.serializers import CSVSerializer

csv_serializer = CSVSerializer()


client = boto3.client('sagemaker')
baseline = "[[EOS]]"
client.create_endpoint_config(
    EndpointConfigName='explain-classify-malicious-contract',
    ExplainerConfig={
        'ClarifyExplainerConfig': {
            "InferenceConfig": {"FeatureTypes": ["text"]},
            'EnableExplanations': '`true`',
            'ShapConfig': {
                'ShapBaselineConfig': {
                    "ShapBaseline": csv_serializer.serialize(baseline)
                },
                'NumberOfSamples': 10,
                'TextConfig': {
                    'Granularity': 'token',
                    'Language': 'xx'
                }
            }
        },
    },
    ProductionVariants=[{
        'VariantName': 'AllTraffic',
        'ModelName': 'huggingface-pytorch-inference-2023-09-14-16-52-08-652',
        'InitialInstanceCount': 1,
        'InstanceType': 'ml.m5.xlarge',
    }]
)

client.create_endpoint(
    EndpointName='explain-classify-malicious-contract',
    EndpointConfigName='explain-classify-malicious-contract'
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:910856264954:endpoint/explain-classify-malicious-contract',
 'ResponseMetadata': {'RequestId': 'e87c069a-80dc-4586-b69e-6c5704414c9d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e87c069a-80dc-4586-b69e-6c5704414c9d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Tue, 19 Sep 2023 01:25:07 GMT'},
  'RetryAttempts': 0}}

In [61]:
import pandas as pd
import json

sagemaker_runtime_client = boto3.client("sagemaker-runtime")

test = [["[EOS]"]]

serialized = csv_serializer.serialize(test)

d = {'inputs': ["test"]}
df = pd.DataFrame(data=d)
print(df)
serialized = csv_serializer.serialize(df.to_numpy())
data = {
    "inputs": []
}

print(json.dumps(data))

response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName="explain-classify-malicious-contract",
    ContentType="text/csv",
    Accept="text/csv",
    Body=df,
)

  inputs
0   test
{"inputs": []}


ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value:   inputs
0   test, type: <class 'pandas.core.frame.DataFrame'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object